# 数据集说明

trec06c是一个公开的垃圾邮件语料库，由国际文本检索会议提供，分为英文数据集（trec06p）和中文数据集（trec06c），其中所含的邮件均来源于真实邮件保留了邮件的原有格式和内容,垃圾邮件原始数据集来自：https://plg.uwaterloo.ca/~gvcormac/treccorpus06/

由于数据集分散在各个文件中，为了方便我将正样本和负样本分别放在了ham_data和spam_data文件夹中(处女座的强迫症)

正样本数：21766

负样本数：42854

中文停用词：chinese_stop_vocab.txt

# 实现思路

1. 对单个邮件进行数据预处理
    - 去除所有非中文字符，如标点符号、英文字符、数字、网站链接等特殊字符
    - 对邮件内容进行分词处理
    - 过滤停用词

2. 创建特征矩阵和样本数据集
    - feature_maxtrix:shape=(samples, feature_word_nums)
    - leabel; shape = (samples, 1)
    - 词向量的选择：索引或word2vect,注意二者的区别
    
3. 拆分数据集：训练数据集、测试数据集和验证数据集

4. 选择模型，这里选择svm

5. 训练、测试、调参

# 具体实现过程

## 所用到的库

In [37]:
import os
import jieba
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from scipy.stats import uniform

## 将邮件转换为特征词矩阵类

In [38]:
class EmailToWordFeatures:
    '''
    功能:将邮件转换为特征词矩阵
    整个过程包括：
    - 对邮件内容进行分词处理
    - 去除所有非中文字符，如标点符号、英文字符、数字、网站链接等特殊字符
    - 过滤停用词
    - 创建特征矩阵
    '''
    def __init__(self,stop_word_file=None,features_vocabulary=None):
        
        self.features_vocabulary = features_vocabulary
        
        self.stop_vocab_dict = {}  # 初始化停用词
        if stop_word_file is not None:
            self.stop_vocab_dict = self._get_stop_words(stop_word_file)
    
    def text_to_feature_matrix(self,words,vocabulary=None,threshold =10):
        cv = CountVectorizer()
        if vocabulary is None:
            cv.fit(words)
        else:
            cv.fit(vocabulary)
        words_to_vect = cv.transform(words)
        words_to_matrix = pd.DataFrame(words_to_vect.toarray())  # 转换成索引矩阵
        print(words_to_matrix.shape)

        # 进行训练特征词选择，给定一个阈值，当单个词在所有邮件中出现的次数的在阈值范围内时及选为训练特征词、
        selected_features = []
        selected_features_index = []
        for key,value in cv.vocabulary_.items():
            if words_to_matrix[value].sum() >= threshold:  # 词在每封邮件中出现的次数与阈值进行比较
                selected_features.append(key)
                selected_features_index.append(value)
        words_to_matrix.rename(columns=dict(zip(selected_features_index,selected_features)),inplace=True)
        return words_to_matrix[selected_features]

    def get_email_words(self,email_path, max_email = 600):
        '''
        由于机器配置问题，作为测试给定阈值600，及正负样本数各位600
        '''
        self.emails = email_path
        if os.path.isdir(self.emails):
            emails = os.listdir(self.emails)
            is_dir = True
        else:
            emails = [self.emails,]
            is_dir = False
            
        count = 0
        all_email_words = []
        for email in emails:
            if count >= max_email:  # 给定读取email数量的阈值
                break
            if is_dir:
                email_path = os.path.join(self.emails,email)
            email_words = self._email_to_words(email_path)
            all_email_words.append(' '.join(email_words))
            count += 1
        return all_email_words
        
    def _email_to_words(self, email):
        '''
        将邮件进行分词处理，去除所有非中文和停用词
        retrun:words_list
        '''   
        email_words = []
        with open(email, 'rb') as pf:
            for line in pf.readlines():
                line = line.strip().decode('gbk','ignore')
                if not self._check_contain_chinese(line):  # 判断是否是中文
                    continue
                word_list = jieba.cut(line, cut_all=False)  # 进行分词处理
                for word in word_list:
                    if word in self.stop_vocab_dict or not self._check_contain_chinese(word):
                        continue  # 判断是否为停用词
                    email_words.append(word)
            return email_words
      
    def _get_stop_words(self,file):
        '''
        获取停用词
        '''
        stop_vocab_dict = {}
        with open(file,'rb') as pf:
            for line in pf.readlines():
                line = line.decode('utf-8','ignore').strip()
                stop_vocab_dict[line] = 1
        return stop_vocab_dict
    
    def _check_contain_chinese(self,check_str):
        '''
        判断邮件中的字符是否有中文
        '''
        for ch in check_str:
            if u'\u4e00' <= ch <= u'\u9fff':
                return True
        return False

## 将正负邮件数据集转换为词特征列表，每项为一封邮件

In [39]:
index_file= '/root/autodl-tmp/trec06c_SMS_SVM/datasets/trec06c/full/index'
stop_word_file = '/root/autodl-tmp/trec06c_SMS_SVM/datasets/trec06c/chinese_stop_vocab.txt'
ham_file = '/root/autodl-tmp/trec06c_SMS_SVM/datasets/trec06c/ham_data'
spam_file = '/root/autodl-tmp/trec06c_SMS_SVM/datasets/trec06c/spam_data'

In [40]:
email_to_features = EmailToWordFeatures(stop_word_file=stop_word_file)
ham_words = email_to_features.get_email_words(ham_file)
spam_words = email_to_features.get_email_words(spam_file)
print('ham email numbers:',len(ham_words))
print('spam email numbers:',len(spam_words))

ham email numbers: 600
spam email numbers: 600


## 将所有邮件转换为特征词矩阵，及模型输入数据

In [41]:
all_email = []
all_email.extend(ham_words)
all_email.extend(spam_words)
print('all test email numbers:',len(all_email))
words_to_matrix = email_to_features.text_to_feature_matrix(all_email)
print(words_to_matrix)

all test email numbers: 1200
(1200, 22575)
      故事  领导  回到  儿子  感情  有个  大概  民俗  出国  教育  ...  培训网  商友会  网管  埃森哲  驱鼠器  条例  \
0      1   2   1   1   1   1   1   1   1   1  ...    0    0   0    0    0   0   
1      0   0   0   0   5   0   0   0   0   0  ...    0    0   0    0    0   0   
2      0   0   0   0   0   0   0   0   0   0  ...    0    0   0    0    0   0   
3      0   0   0   0   0   0   0   0   0   0  ...    0    0   0    0    0   0   
4      0   0   0   0   0   0   0   0   0   0  ...    0    0   0    0    0   0   
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ..  ...  ...  ..   
1195   0   0   0   0   0   0   0   0   0   0  ...    0    0   0    0    0   0   
1196   0   0   0   0   0   0   0   0   0   0  ...    0    0   0    0    0   0   
1197   0   0   0   0   0   0   0   0   0   0  ...    0    0   0    0    0   0   
1198   0   0   0   0   0   0   0   0   0   0  ...    0    0   0    0    0   0   
1199   0   0   0   0   0   0   0   0   0   0  ...    0    0   0   

## 获取标签矩阵

In [42]:
label_matrix = np.zeros((len(all_email),1))
label_matrix[0:len(ham_words),:] = 1 

# 使用svm模型进行训练

In [43]:
# 拆分数据集
x_train,x_test,y_train,y_test = train_test_split(words_to_matrix,label_matrix,test_size=0.2,random_state=42)

# 使用LinearSVC模型进行训练
svc = LinearSVC(loss='hinge',dual=True)
param_distributions = {'C':uniform(0,10)}
rscv_clf =RandomizedSearchCV(estimator=svc, param_distributions=param_distributions,cv=3,n_iter=200,verbose=2)
rscv_clf.fit(x_train,y_train)
print('best_params:',rscv_clf.best_params_)

Fitting 3 folds for each of 200 candidates, totalling 600 fits
[CV] END ................................C=8.453789488711907; total time=   0.1s
[CV] END ................................C=8.453789488711907; total time=   0.1s


/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END ................................C=8.453789488711907; total time=   0.1s
[CV] END ................................C=6.072832147425263; total time=   0.1s
[CV] END ................................C=6.072832147425263; total time=   0.1s


/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ................................C=6.072832147425263; total time=   0.1s
[CV] END ...............................C=0.7288839773791123; total time=   0.1s
[CV] END ...............................C=0.7288839773791123; total time=   0.1s


...

/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END ................................C=8.746421416051252; total time=   0.1s
[CV] END ................................C=8.746421416051252; total time=   0.1s
[CV] END ................................C=8.746421416051252; total time=   0.1s


/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] END ................................C=2.187474881676219; total time=   0.1s
[CV] END ................................C=2.187474881676219; total time=   0.1s
[CV] END ................................C=2.187474881676219; total time=   0.1s
best_params: {'C': 0.025174442128299557}


/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/miniconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [44]:
# 使用测试数据集进行测试
y_prab = rscv_clf.predict(x_test)
print('accuracy:',accuracy_score(y_prab,y_test))

accuracy: 0.9833333333333333


# 分别选择一封正式邮件和垃圾邮件进行

<b>正式邮件内容如下：</b>

- 上海同捷数字化技术有限公司是上海同济同捷科技股份有限公司下属的高科技公司，于2003年8月正式成立。
前身为美国UGS公司中国研发中心和上海同济同捷科技股份有限公司软件研发中心。
为美国UGS公司研发UG软件(三维设计软件)和同捷科技股份IPDM产品、IDQC产品。为通用（GM）、福特（Ford）、伟世通（Visteon）等开发工程应用软件。
本公司是国内从事技术信息化和管理信息化开发和服务的最大公司之一，主要产品有PDM、IDQC和DPS系统，集团公司是国内唯一能从事汽车设计的专业公司，共有员工600人，公司位置位于浦东金桥高新技术开发区;
工作地点:北京(公司北京的研发分部)
C++程序员：
1)      本科以上学历，包括本科；一年以上开发经验；
2)      熟悉Windows编程，精通Visual C++ 6.0和MFC类库；
3)      熟练使用Oracle和MS SqlServer，精通SQL；
4)      精通并熟练使用COM/ActiveX技术；
5)      能够独立根据设计文档进行编码工作；
6)      具备良好的沟通能力；
7)      具有机械制造知识者优先；

<b>垃圾邮件如下：</b>

- 中国创业无忧网http://www.007edu.com为创业者提供创业一条龙服务，主要服务有：代写商业文书、创业培训、自编创业书籍销售、创业融资、创业招商。
代写商业文书主要业务如下：

◆商务文书——商业计划书、项目策划书、商务标书、商务活动策划书、招商文案、商业信函等。 

◆管理文书——规章制度、工作流程、职位说明书、企业全套管理方案等。 

◆广告文案——企业简介、企业手册、产品手册、软性宣传文章、企业内/外报、企业简报等。 

◆各类公文——计划、总结、请示、报告、专题演讲、竞职演讲、就职演说、各类致辞、会议纪要、会议发言、心得体会、业绩材料等。 

◆法律文书——合同、协议、起诉状、答辩状、执行申请等。 

◆网站文案——网络推广方案、网络营销方案、互联网宣传文章 、互联网广告文案等。 

◆其它文稿——毕业论文、职称论文、其它各类专业论文、工作通讯、人物传记、剧本创作等。 

欢迎咨询，电话：027-61056811 QQ：357336084 

详情请浏览：http://www.007edu.com/jhs.asp

---------------------- http://etide.51.net/ ---------------------
以上内容由VolleyMail免费软件发送，但内容与VolleyMail无关，特此声明！


In [45]:
def email_to_predict_matrix(words,features):
    cv = CountVectorizer()
    words_to_vect = cv.fit_transform(words)
    words_to_marix = pd.DataFrame(words_to_vect.toarray())
    vocabulary = cv.vocabulary_
    
    words_numbers_list = [] # 特征词出现的次数列表
    for feature in features:
        if feature in cv.vocabulary_.keys():
            words_numbers_list.append(words_to_marix[vocabulary[feature]][0])
        else:
            words_numbers_list.append(0)
    words_numbers_matrix = pd.DataFrame([words_numbers_list],columns = features)
    return words_numbers_matrix

In [46]:

valid_ham_email = '/root/autodl-tmp/trec06c_SMS_SVM/datasets/trec06c/001_010'
valid_spam_email = '/root/autodl-tmp/trec06c_SMS_SVM/datasets/trec06c/000_134'

email_to_features_valid = EmailToWordFeatures(stop_word_file=stop_word_file)
valid_ham_email_words = email_to_features_valid.get_email_words(valid_ham_email)
valid_spam_email_words = email_to_features_valid.get_email_words(valid_spam_email)

valid_ham_words_maxtrix = email_to_predict_matrix(valid_ham_email_words,words_to_matrix.columns)
valid_spam_words_maxtrix = email_to_predict_matrix(valid_spam_email_words,words_to_matrix.columns)

In [47]:
print('测试正式邮件----------')
print('预测结果：',rscv_clf.predict(valid_ham_words_maxtrix))

测试正式邮件----------
预测结果： [1.]


In [48]:
print('测试垃圾邮件----------')
print('预测结果：',rscv_clf.predict(valid_spam_words_maxtrix))

测试垃圾邮件----------
预测结果： [0.]
